# Parameters

In [ ]:
iterations = 1 # number of something
epochs_step = 10 # number of something

# Preparation

## Install Requirements

In [ ]:
!nvidia-smi

Thu Jun 20 05:42:33 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   51C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
#!pip install -U ultralytics
!pip install ultralytics==8.0.196

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="nuI13APqQA4qc063c07u")
project = rf.workspace("giolor").project("robotics_project_dataset")
version = project.version(1)
dataset = version.download("yolov8")

loading Roboflow workspace...
loading Roboflow project...


## Install YOLOv8

In [ ]:
from ultralytics import YOLO
import os
from IPython.display import display, Image
from IPython import display
display.clear_output()
#!yolo mode=checks

# Train YOLOv8 Model on Custom Dataset

In [ ]:
def get_epochs(file):
  if file == None:
    raise Exception("Missing counter path")

  counter_file = open(file, "r")
  counter = int(counter_file.readline())
  counter_file.close()
  return counter

def set_epochs(file, epochs):
  if file == None:
    raise Exception("Missing counter path")
  elif epochs == None:
    raise Exception("Missing epochs")

  counter_file = open(f"{drive_folder}/counter.txt", "w")
  counter_file.write(f"{epochs}")
  counter_file.close()

In [ ]:
import os
base_path = '/content/runs/detect'
drive_folder = "/content/drive/MyDrive/robotics_project_model"
counter_path = f"{drive_folder}/counter.txt"

epochs_session = 0
epochs_total = get_epochs(counter_path)

for i in range(0,iterations):

  print("\n")
  print("**********************************\n")
  print(f"Session epochs: {epochs_session}\n")
  print(f"Total epochs: {epochs_total}\n")
  print("**********************************\n")

  # Load a model
  model_path = f"{drive_folder}/train/weights/best.pt"
  if not os.path.isfile(model_path):
    model_path = f"{drive_folder}/old/best.pt"

  model = YOLO(model_path)  # load a pretrained model (recommended for training)

  # Save previous model
  os.system(f"cp {drive_folder}/train/weights/best.pt {drive_folder}/old/best.pt")
  os.system(f"rm -rf {drive_folder}/train")
  os.system(f"mv {drive_folder}/val {drive_folder}/old")

  # Use the model
  model.train(data=f"{drive_folder}/data-roboflow.yaml", epochs=epochs_step, project=f"{drive_folder}", name="train")  # train the model
  model.val(data=f"{drive_folder}/data-roboflow.yaml", project=f"{drive_folder}", name="val")  # evaluate model performance on the validation set


  #path = model.export(format="onnx")  # export the model to ONNX format

  epochs_session += epochs_step
  epochs_total += epochs_step
  set_epochs(counter_path, epochs_total)



**********************************

Session epochs: 0

Total epochs: 957

**********************************



New https://pypi.org/project/ultralytics/8.2.36 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.196 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=/content/drive/MyDrive/robotics_project_model/old/best.pt, data=/content/drive/MyDrive/robotics_project_model/data-roboflow.yaml, epochs=10, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=/content/drive/MyDrive/robotics_project_model, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=F